In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import collections
import numpy as np
from spacy.en import English
import string
import unicodedata
import gensim
import re
from collections import Counter
from itertools import chain
from scipy.spatial import distance
import spacy
from collections import Counter

nlp = spacy.load('en')
parser = English()


sqd_data = pd.read_json("/home/anjana/Downloads/train-v1.1.json") 


In [2]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/home/anjana/Downloads/GoogleNews-vectors-negative300.bin', binary=True)


In [108]:
stopword = stopwords.words('english')
qtntype_list = ['why','how', 'what', 'when','which', 'who','whom', 'where','whose']
pun_list = list(string.punctuation)
delelements_list = stopword + qtntype_list + pun_list

In [109]:
def replace_all(repls, str):
   # return re.sub('|'.join(repls.keys()), lambda k: repls[k.group(0)], str)                                    
   return re.sub('|'.join(re.escape(key) for key in repls.keys()),
                 lambda k: repls[k.group(0)], str)

In [110]:
def preprocess_question(para_question):
    question_list = []
    for i in range(len(para_question)):
        l1 = para_question[i]['question']
        l1 = unicodedata.normalize('NFKD', l1).encode('ascii','ignore').decode('utf-8')
        question_list.append(l1)
    
    #Appropriate question type replacement
    Question_list = []
    for i in range(len(question_list)):
        Question_list.append(replace_all({"how": "how", "when": "when","what": "what","whom": "whom","which": "which", "where": "where", "who":"who" ,"whose":"whose"}, question_list[i]))
        
    return Question_list

In [111]:
def preprocess_text(para_context):

    context = unicodedata.normalize('NFKD', para_context).encode('ascii','ignore').decode('utf-8')
    #print(context)
    #sentence_tokeniztion
    senttokenized_context = sent_tokenize(context)
    sentencelist = []
    for sent_token in senttokenized_context:
        sentencelist.append(sent_token) 
    
    sentence_list = []
    lemmatizer = WordNetLemmatizer()


    for i in sentencelist:
        words = nltk.word_tokenize(i)
        words = ([x.lower() for x in words])
        words = ([x for x in words if x not in delelements_list])
        words = [lemmatizer.lemmatize(x) for x in words]
        words = [[s.encode('ascii').decode('utf-8') for s in words]]
        sentence_list.append(words)
    return sentencelist, sentence_list

In [112]:
def question_type_extractor(question_mapping):
    Question_mapping = []
    for i in range(len(question_mapping)):
        if question_mapping[i] != 'how many' and question_mapping[i] != 'how much':
            Question_mapping.append(question_mapping[i].split()[0])
        else:
             Question_mapping.append(question_mapping[i])
    return Question_mapping


In [113]:
def wmd_que_ans_pair(que_list, sen_list):
    wmd_matrix = np.zeros([len(que_list),len(sen_list)]) 
    for i in range(len(que_list)):
        for j in range(len(sen_list)):
            wmd_matrix[i,j] =  word2vec_model.wmdistance(que_list[i],sen_list[j])  
    wmd_df = pd.DataFrame(wmd_matrix)
    wmd_df['sentence'] = wmd_df.T.idxmin()
    wmd_df['qnno'] = wmd_df.index
    que_no_ans_sen_no = []
    for i in range(len(wmd_df)):
        que_no_ans_sen_no.append([wmd_df['qnno'][i], wmd_df['sentence'][i]])
    qn_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        if que_no_ans_sen_no[i][0] == que_list.index(que_list[i]):
            qn_sentence.append(que_list[i])
    an_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        for j in range(len(sen_list)):
            if que_no_ans_sen_no[i][1] == sen_list.index(sen_list[j]):
                an_sentence.append(sen_list[j]) 
    return qn_sentence, an_sentence

In [114]:
def que_ner_ans_ner(que_sentence, ans_sentence):
    ans_sentence_mapping = []
    for i in range(len(ans_sentence)):
        doc_ans = nlp(ans_sentence[i])
        for ent in doc_ans.ents:
            ans_sentence_mapping.append([i,ent.label_])

    que_sentence_mapping = []
    for i in range(len(que_sentence)):
        doc_que = nlp(que_sentence[i])
        for ent in doc_que.ents:
            que_sentence_mapping.append([i,ent.label_])
    return que_sentence_mapping, ans_sentence_mapping

In [10]:
def que_df_ans_df(que_sen_map, ans_sen_map):
    qno_anstag_df = pd.DataFrame(ans_sen_map,columns = ['QuestionNo','AnswerTag'])
    qno_quetag_df = pd.DataFrame(que_sen_map, columns = ['QuestionNo', 'QuestionTag'])
    return qno_quetag_df, qno_anstag_df



In [11]:
def que_ans_combo_(questionword, answerword):
    ques_ans_combo = []
    for i in range(len(questionword)):
        for j in range(len(answerword)):
            ques_ans_combo.append([questionword[i],answerword[j]])
    return(ques_ans_combo)


In [12]:
def question_type(process_que_list):
    question_mapping = [] 
    for i in range(len(process_que_list)):
        l2 = nltk.word_tokenize(process_que_list[i])
        #print(len(l2))
        #print(l2)
        missed = l2[-1]        
        if missed == '?':
            for j in range(len(l2)):


                if l2[j] in qtntype_list:
                    combine_list = l2[j]+" "+l2[j+1]
                    question_mapping.append(combine_list)

        else:

             for k in range(len(l2)):#8

                if k < len(l2)-1:  #7        

                    if l2[k] in qtntype_list:
                        combine_list = l2[k]+" "+l2[k+1]
                        question_mapping.append(combine_list) 
                else:

                        if l2[k] in qtntype_list:
                            combine_list = l2[k]+" "+'abc'
                            question_mapping.append(combine_list) 
    return question_mapping


In [13]:
def qntypefun(qnlist):
    punc=set(",./;'?&-")

    common_list = []
    for i in qnlist:
        list1 = (i.lower().split())
        list2 = list(set(list1).intersection(qtntype_list)) 
        for j in list2:
            strp =''.join(c for c in j if not c in punc)
            common_list.append(strp) 
        if list2 == []:
            common_list.append('others')
        
        
    return common_list

In [14]:
co_occ_matrix_df = pd.read_csv("/home/anjana/co_occ_matrix_final", header=0, index_col=0)
co_occ_matrix = co_occ_matrix_df.values
co_occ_matrix

array([[  385,   360,    12,    81,     8,    29,    30,    21,   421,
           26,    13,    11,    65,   230,   577,   589],
       [ 3047,  2574,   259,   774,    76,   234,   264,   113,  4630,
          224,   722,   653,   777,  6862,  5670,  5829],
       [19765, 15133,  1416,  4367,   435,  1246,  1979,   791, 18870,
          622,  2112,   827,  3896, 12288, 28041, 31681],
       [ 3206,  2266,   182,   570,    58,   257,   302,    84,  4718,
          102,    56,    84,   810,  1659,  4272,  4559],
       [ 1959,  1188,   291,   536,    35,   105,   140,    40,  1371,
           31,    39,    35,   389,   947,  3366,  2944],
       [ 8106,  3404,   230,   673,    90,   286,   513,   102,  4063,
           92,    88,   102,   955,  1903,  5250,  6888],
       [ 3886,  2290,   338,   835,    92,   199,   369,   104,  2779,
           84,   145,   130,   671,  1749,  4917,  5023],
       [  201,    79,     8,    13,     1,    13,     4,     3,   111,
            1,     1,     

In [15]:
row_list = ['why','how','what','when','where','who', 'which','whom','whose','others','PERSON',
          'NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
          'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE', 'ORG']


column_list = ['PERSON','NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
               'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE','ORG']

In [16]:
co_occ_matrix_df['Tags'] = co_occ_matrix_df.index

from sklearn.preprocessing import normalize
normed_matrix = normalize(co_occ_matrix, axis=1, norm='l1') 
normed_matrix_df = pd.DataFrame(normed_matrix, index = row_list, columns= column_list)


/home/anjana/nltk/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [17]:
normed_matrix_df['Tags'] = normed_matrix_df.index

#normalized heatmap
melt_normed_matrix_df = pd.melt(normed_matrix_df,id_vars = ['Tags'])
melt_normed_matrix_df.head()

,Tags,variable,value
0,why,PERSON,0.134710
1,how,PERSON,0.093158
2,what,PERSON,0.137765
3,when,PERSON,0.138279
4,where,PERSON,0.146020


In [18]:
new_normed_matrix  = 1 - normed_matrix
new_normed_matrix_df = pd.DataFrame(new_normed_matrix, index = row_list, columns= column_list)
new_normed_matrix_df['Tags'] = new_normed_matrix_df.index

In [19]:
melt_new_normed_matrix_df = pd.melt(new_normed_matrix_df,id_vars = ['Tags'])
melt_new_normed_matrix_df.head()

,Tags,variable,value
0,why,PERSON,0.865290
1,how,PERSON,0.906842
2,what,PERSON,0.862235
3,when,PERSON,0.861721
4,where,PERSON,0.853980


In [20]:
answer_dict = {'LOC': 1,'PERSON': 2,'NORP': 3,'FAC': 4,'PRODUCT': 5,'EVENT': 6,'WORK_OF_ART': 7,'LANGUAGE': 8,'DATE':9,'TIME': 10,'PERCENT': 11,'MONEY': 12,'ORDINAL': 13,'CARDINAL': 14,'GPE': 15, 'ORG': 16}
question_dict = {'LOC': 10,
                 'PERSON': 20,
                 'NORP': 30,
                 'FAC': 40,
                 'PRODUCT': 50,
                 'EVENT':60,
                 'WORK_OF_ART':70,
                 'LANGUAGE':80,
                 'DATE': 90,
                 'TIME': 100,
                 'PERCENT': 110,
                 'MONEY': 120,
                 'ORDINAL':130,
                 'CARDINAL': 140,
                 'GPE': 150,
                 'ORG': 160,
                 
                 'what': 170,
                 'when': 180,
                 'where': 190,
                 'who': 200,
                 'which':210,
                 'whom': 220,
                'whose': 230,
                'how':240,
                'why':250,
                'others':260}

In [21]:
melt_new_normed_matrix_df_1 = melt_new_normed_matrix_df.copy()
melt_new_normed_matrix_df_1["Tags"].replace(question_dict, inplace=True)
melt_new_normed_matrix_df_1['variable'].replace(answer_dict, inplace = True)

In [22]:
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy, Reader
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))

In [23]:
csv_file_1 = melt_new_normed_matrix_df_1.to_csv("new_melt.txt", sep= '\t', header = False, index = False)

In [24]:
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))
melt_1 = Dataset.load_from_file("/home/anjana/new_melt.txt", reader_new)
print(melt_1)

In [25]:
import math
from sklearn.metrics import mean_squared_error

In [26]:
reconstruction_error_1 = []
for i in range(20):
    
    algo = SVDpp(n_factors=i)
    
    for trainset, testset in melt_1.folds():
        algo.train(trainset)
        predictions_svdpp = algo.test(testset)
        
    
    reconstruct = np.dot(algo.pu, algo.qi.T)
    rmse = mean_squared_error(new_normed_matrix, reconstruct)**0.5
    reconstruction_error_1.append(rmse)
    print(i, rmse)

0 0.94047868429
1 0.942213873501
2 0.940502347756
3 0.940639200582
4 0.940671790955
5 0.941029764317
6 0.941641171764
7 0.938803150419
8 0.94073593691
9 0.93988850626
10 0.940175135038
11 0.94266592945
12 0.943256640538
13 0.940844166272
14 0.944951000107
15 0.938362767502
16 0.943004207306
17 0.941106863366
18 0.939660811662
19 0.942797676387


In [27]:
algo = SVDpp(n_factors=10)
    
for trainset, testset in melt_1.folds():
    algo.train(trainset)
    predictions_svdpp = algo.test(testset)
    svdpp_df = pd.DataFrame(predictions_svdpp)

In [28]:
svdpp_df['error'] = svdpp_df.r_ui - svdpp_df.est #error =  observed - estimated
svdpp_df.head()

,uid,iid,r_ui,est,details,error
0,250,3,0.874038,0.879043,{'was_impossible': False},-0.005006
1,30,7,0.990133,0.956077,{'was_impossible': False},0.034056
2,20,16,0.833737,0.838746,{'was_impossible': False},-0.005009
3,50,10,0.996463,1.000000,{'was_impossible': False},-0.003537
4,80,11,0.989458,0.959966,{'was_impossible': False},0.029492


In [29]:
reconstruct = np.dot(algo.pu, algo.qi.T) #reconstructed matrix
rmse = mean_squared_error(new_normed_matrix, reconstruct)**0.5

In [30]:
reconstruct_error = new_normed_matrix - reconstruct
reconstruct_error_df = pd.DataFrame(reconstruct_error,index = row_list,columns = column_list)


In [31]:
row_column_list =['why',
 
  'how',
 'whose',               
 'what',
 'when',
 'where',
 'who',
 'which',
 'whom',
'others',
                
                  
 'q_PERSON',
 'q_NORP',
 'q_FAC',
 'q_LOC',
 'q_PRODUCT',
 'q_EVENT',
 'q_WORK_OF_ART',
 'q_LANGUAGE',
 'q_DATE',
 'q_TIME',
 'q_PERCENT',
 'q_MONEY',
 'q_ORDINAL',
 'q_CARDINAL',
 'q_GPE',
 'q_ORG',
 'a_PERSON',
 'a_NORP',
 'a_FAC',
 'a_LOC',
 'a_PRODUCT',
 'a_EVENT',
 'a_WORK_OF_ART',
 'a_LANGUAGE',
 'a_DATE',
 'a_TIME',
 'a_PERCENT',
 'a_MONEY',
 'a_ORDINAL',
 'a_CARDINAL',
 'a_GPE',
 'a_ORG']

In [32]:
qn_ans_list = list(algo.pu) + list(algo.qi)
qn_ans_list

qn_ans_list_copy = qn_ans_list.copy()
len(qn_ans_list_copy)

42

In [33]:
question_answer_dis = np.zeros((len(row_column_list), len(row_column_list))) 

for i in range(len(qn_ans_list)):
    for j in range(len(qn_ans_list_copy)):
        question_answer_dis[i,j] = distance.euclidean(qn_ans_list[i],qn_ans_list_copy[j])

In [34]:
eucli_matrix_df = pd.DataFrame(question_answer_dis, index = row_column_list, columns= row_column_list)
eucli_matrix_df

,why,how,whose,what,when,where,who,which,whom,others,...,a_WORK_OF_ART,a_LANGUAGE,a_DATE,a_TIME,a_PERCENT,a_MONEY,a_ORDINAL,a_CARDINAL,a_GPE,a_ORG
why,0.000000,0.600799,0.435066,0.366898,0.611132,0.402925,0.424521,0.357173,0.491603,0.462207,...,0.396935,0.557371,0.442841,0.418088,0.399648,0.519318,0.592583,0.521372,0.356712,0.423624
how,0.600799,0.000000,0.417528,0.389430,0.600678,0.654553,0.373058,0.510564,0.374561,0.544260,...,0.392664,0.229448,0.434955,0.478890,0.396747,0.485133,0.337167,0.323096,0.386563,0.384252
whose,0.435066,0.417528,0.000000,0.245289,0.541527,0.384160,0.177534,0.525601,0.320184,0.306609,...,0.223637,0.258513,0.373099,0.248293,0.304413,0.305677,0.357052,0.241060,0.311427,0.292264
what,0.366898,0.389430,0.245289,0.000000,0.555047,0.375642,0.251462,0.460632,0.211500,0.435361,...,0.240732,0.276266,0.304846,0.240059,0.329221,0.447032,0.372610,0.370307,0.355939,0.292311
when,0.611132,0.600678,0.541527,0.555047,0.000000,0.448625,0.489957,0.490314,0.602603,0.629173,...,0.460491,0.565899,0.429988,0.558715,0.483158,0.661555,0.442643,0.508787,0.600091,0.467487
where,0.402925,0.654553,0.384160,0.375642,0.448625,0.000000,0.411259,0.492386,0.519713,0.537839,...,0.338974,0.516055,0.395376,0.408953,0.477848,0.558493,0.538604,0.482871,0.529264,0.457535
who,0.424521,0.373058,0.177534,0.251462,0.489957,0.411259,0.000000,0.488138,0.286581,0.409549,...,0.184626,0.253145,0.296926,0.326053,0.273828,0.351316,0.341228,0.275252,0.331849,0.156695
which,0.357173,0.510564,0.525601,0.460632,0.490314,0.492386,0.488138,0.000000,0.598819,0.581174,...,0.438995,0.537163,0.431248,0.492355,0.491129,0.608726,0.526378,0.463722,0.414295,0.483921
whom,0.491603,0.374561,0.320184,0.211500,0.602603,0.519713,0.286581,0.598819,0.000000,0.468860,...,0.322156,0.297634,0.391289,0.348685,0.295492,0.460033,0.385289,0.440452,0.423173,0.270073
others,0.462207,0.544260,0.306609,0.435361,0.629173,0.537839,0.409549,0.581174,0.468860,0.000000,...,0.441187,0.466525,0.533939,0.351640,0.320422,0.367368,0.488701,0.410238,0.288729,0.438283


In [35]:
def word_count(doc_ner_list):
    count = Counter(doc_ner_list)
    map_count = []
    for i in count.items():
        map_count.append(i)
    return map_count

def bag_of_words(doc_ner_list, unique_ner):
    dis_matrix = np.zeros(len(unique_ner), dtype = float)
    nbow = word_count(doc_ner_list)
   
    doc_len = len(doc_ner_list)
    for i, freq in nbow:
        dis_matrix[unique_ner.index(i)] = freq / float(doc_len)  # Normalized word frequencies.
    return dis_matrix


In [36]:
def ner_combination(map_que_ner, map_ans_ner):
     
    unique_removed_new_ner = list(set(map_que_ner + map_ans_ner))
    return unique_removed_new_ner

In [37]:
from pyemd import emd


In [38]:
def qntypefun(qnlist):
    punc=set(",./;'?&-")

    common_list = []
    for i in qnlist:
        list1 = (i.lower().split())
        list2 = list(set(list1).intersection(qtntype_list)) 
        for j in list2:
            strp =''.join(c for c in j if not c in punc)
            common_list.append(strp) 
        if list2 == []:
            common_list.append('others')
        
        
    return common_list

###### Tag distance ######

In [43]:
len(sqd_data)

442

In [44]:
Tagdistance = [] 

for i in range(len(sqd_data)):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    for k in range(len(paragraph_pointer)):#no of paragraphs in a row of the df

        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])

        

        ls = [] 

        que_type_clean = qntypefun(processed_que_list)

        for m in range(len(que_type_clean)):

            map_que_ner = []
            map_que_ner.append(que_type_clean[m])
            #print("Que_NER", " ", map_que_ner)
            for j in range(len(processed_sen_list)):



                map_ans_ner = []
                ans_ner = nlp(processed_sen_list[j])
                for ent in ans_ner.ents:
                    map_ans_ner.append('a_' + ent.label_)
                #print("Ans_NER", " ", map_ans_ner)
                unique_ner = ner_combination(map_que_ner, map_ans_ner)
                #print("Unique_NER", " ", unique_ner)
                #print()
                # distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
                d_1 = bag_of_words(map_que_ner, unique_ner)
                #print(d_1)

                d_2 = bag_of_words(map_ans_ner, unique_ner)
                #print(d_2)
                sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
                #print(sub_matrix_df)
                sub_matrix= sub_matrix_df.as_matrix()
                sub_matrix[np.isnan(sub_matrix)] = 0

                #print(sub_matrix)
                score = emd(d_1, d_2, sub_matrix)
                #print(processed_sen_list[j])
                #print('Score = ', score)
                #l1 = [m,j,score] 
                #ls.append(pd.DataFrame([m,j,score])
                #tagdis = pd.DataFrame(l1) 
                #tagdis['TagRank'] = tagdis.groupby('Qnno')['Tagdis'].rank(method = "first",ascending=True)
                #Tagdistance.append(ls)
                ls.append([m,j,score]) 
                
                
                
                df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
                df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

        Tagdistance.append(df)
    
    #return Tagdistance
                
                #print([i,k,m,j,score])
                
               # print("****************************************************************************************************")



In [46]:
len(Tagdistance)

18896

In [47]:
Tagdistance[0].head()

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.378392,4.0
1,0,1,0.349630,1.0
2,0,2,0.416877,7.0
3,0,3,0.378679,5.0
4,0,4,0.371880,3.0


##### W2V distance ##### 

In [57]:
w2vdistance = [] 

for i in range(len(sqd_data)):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 
    

    for k in range(len(paragraph_pointer)):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])
        distance = [] 
        for m in range(len(processed_que_list)):
            for j in range(len(processed_sen_list)):

                #word2vec_model.init_sims(replace=True)
                score = word2vec_model.wmdistance(processed_que_list[m].split(), processed_sen_list[j].split())
                score_df = [m,j,score]
                distance.append(score_df)
                distance_df = pd.DataFrame(distance,columns = ['Qnno','Sentno','w2vDistance'])
                distance_df['Rank'] = distance_df.groupby('Qnno')['w2vDistance'].rank(method = "first",ascending=True)
        w2vdistance.append(distance_df)

        


In [58]:
len(w2vdistance)

18896

In [60]:
w2vdistance[0].head()

,Qnno,Sentno,w2vDistance,Rank
0,0,0,2.652292,3.0
1,0,1,2.631091,2.0
2,0,2,2.906310,7.0
3,0,3,2.734407,5.0
4,0,4,2.763077,6.0


In [115]:
def rankfun(df):
    return list(df[df['Rank']==1.0]['Sentno']) 

In [62]:
tagfirstrank = [] 
w2vfirstrank = [] 

for i in range(len(Tagdistance)):
    tag = rankfun(Tagdistance[i])
    w2v = rankfun(w2vdistance[i])
    tagfirstrank.append(tag)
    w2vfirstrank.append(w2v)
    
    

In [64]:
len(w2vfirstrank)


18896

###### Qn-Actual ans ##### 

In [116]:
def answerprocessfun(context):
    answer_ls = [] 
    for i in range(len(context['qas'])):
       
        answer_ls.append(context['qas'][i]['answers'][0]['text'].lower().split())
    return answer_ls


In [117]:
def commonsen(list1,list2):
    return len(set(list1) & set(list2))


In [70]:
qn_anssent = [] 

for b in range(len(sqd_data)):
    #Content_list.append(b)                                 
    
    #Qn_list = [] 
    #Sentence_list = [] 
    paragraph_pointer = sqd_data.data[b]['paragraphs'] 


    for j in range(len(paragraph_pointer)):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[j]['context']) 
        #processed_que_list = preprocess_question_second(paragraph_pointer[j]['qas'])
        que_list = preprocess_question(paragraph_pointer[j]['qas'])
        processed_ans_list = answerprocessfun(paragraph_pointer[j])

        qn_ans_commonwords_list = np.zeros([len(processed_ans_list),len(process_sen_tokenized_list)])
        for i in range(len(processed_ans_list)):
                for j in range(len(process_sen_tokenized_list)):
                    qn_ans_commonwords_list[i,j] = len(set(processed_ans_list[i]) & set(process_sen_tokenized_list[j][0]))
                df = pd.DataFrame(qn_ans_commonwords_list)
                df['AnsSent'] = df.T.idxmax()
                df['Qnno'] = df.index 
        qn_anssent.append(df[['Qnno','AnsSent']])

In [144]:
len(qn_anssent)

18896

In [72]:
actual_anssent = [] 
for i in range(len(qn_anssent)):
    actual_anssent.append(list(qn_anssent[i]['AnsSent'])) 

In [73]:
actual_anssent[0]

[5, 2, 1, 4, 1]

In [118]:
def zeronemapping(list1,list2):
    mappinglist = [] 
    for i in range(len(list1)):
        if (list1[i]==list2[i]):
            mappinglist.append(1)
        else:
            mappinglist.append(0)
    return mappinglist

In [75]:
tag_mapping = [] 
w2v_mapping = [] 
for i in range(len(w2vfirstrank)):
    w2v_mapping.append(zeronemapping(actual_anssent[i],w2vfirstrank[i])) 
    tag_mapping.append(zeronemapping(actual_anssent[i],tagfirstrank[i])) 

In [76]:
print(len(w2v_mapping))
print(len(tag_mapping))

18896
18896


###### Total Sentence number ###### 

In [77]:
qn_actualans_df = pd.concat(qn_anssent)
totalsen_no = len(qn_actualans_df)
totalsen_no

87599

In [78]:
w2v_accuracy = (sum([sum(l) for l in w2v_mapping])/totalsen_no)*100
tag_accuracy = (sum([sum(l) for l in tag_mapping])/totalsen_no)*100

In [80]:
print("w2v_distance accuracy", " :", w2v_accuracy,"%")
print("tag_distance accuracy",":",tag_accuracy,"%") 

w2v_distance accuracy  : 60.427630452402425 %
tag_distance accuracy : 22.247970867247343 %


##### Weight distance ######

In [242]:
weights = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

* if the corresponding elements in list1 and list2 are same, we are assigning 1, otherwise 0. (For actual answer sentence and Top one ranked sentence mapping.)

In [119]:
def toponemapping(list1,list2):
    ls = []
    for i in range(len(list1)):
        if (list1[i]==list2[i]): 
            ls.append(1) 
        else:
            ls.append(0)
            
    return ls

* if the elements in list0 includes in list1 / list2 / list3 we are assigning 1, otherwise 0 (actual sentence vs Top three ranked sentence mapping) 
* if the elements in list0 includes in list1 / list2, we are assigning 1, otherwise 0 (For actual sentence vs Top two ranked sentence mapping)

In [244]:
def topthreemapping(list0,list1,list2,list3):
    if (list2==[]):
        list2 = ['nan']*len(list1)
    if (list3==[]):
        list3 = ['nan']*len(list1) 
    intopthree = []
    intoptwo = [] 
    for i in range(len(list1)):
        if (list0[i]==list1[i]) or (list0[i]==list2[i] or (list0[i]==list3[i])):
            intopthree.append(1)
        else:
            intopthree.append(0)
            
        if (list0[i]==list1[i]) or (list0[i]==list2[i]): 
            intoptwo.append(1)
        else:
            intoptwo.append(0)
            
    return intopthree,intoptwo


##### First rank, Second rank, Third rank  sentence  list across all weights ##### 

In [261]:
First_rank_list = [] 
Second_rank_list = []
Third_rank_list = [] 
Distance_list = [] 

for i in range(len(w2vdistance)):
    first_rank = [] 
    second_rank = [] 
    third_rank = [] 
    weighted_distance_list = [] 

    for a in range(len(weights)):

        weigDistance = pd.DataFrame((w2vdistance[i]['w2vDistance']*weights[a]) + (Tagdistance[i]['TagDistance']*(1-weights[a])),columns = ['WeigDistance']) 
        weigDistance['Qnno'] = w2vdistance[i]['Qnno'] 
        weigDistance['Sentno'] =  w2vdistance[i]['Sentno']
        weigDistance['WeiRank'] = weigDistance.groupby('Qnno')['WeigDistance'].rank(method = "first",ascending=True)
        first_rank.append(list(weigDistance[weigDistance['WeiRank']==1.0]['Sentno'])) 
        second_rank.append(list(weigDistance[weigDistance['WeiRank']==2.0]['Sentno']))
        third_rank.append(list(weigDistance[weigDistance['WeiRank']==3.0]['Sentno'])) 
        weighted_distance_list.append(weigDistance)


        
    First_rank_list.append(first_rank) 
    Second_rank_list.append(second_rank)
    Third_rank_list.append(third_rank)
    Distance_list.append(weighted_distance_list) 
    

In [262]:
len(Distance_list)

18896

,WeigDistance,Qnno,Sentno,WeiRank
0,0.605782,0,0,4.0
1,0.577776,0,1,2.0
2,0.665820,0,2,7.0
3,0.614252,0,3,6.0
4,0.611000,0,4,5.0
5,0.564688,0,5,1.0
6,0.584195,0,6,3.0
7,0.625389,1,0,3.0
8,0.649016,1,1,5.0
9,0.678143,1,2,7.0


##### Zero - One mapping for Top one , Top two, Top three ranked sentences #####

In [245]:
topfirstrank_weig_list = [] 
toptworank_weig_list = []
topthreerank_weig_list = [] 

for p in range(len(weights)):

    first_rank_mapping = []
    toptwo_rank_mapping = []
    topthree_rank_mapping = [] 
    for i in range(len(actual_anssent)):
        first_rank_mapping.append(toponemapping(actual_anssent[i],First_rank_list[i][p]))
        topthree,toptwo = topthreemapping(actual_anssent[i],First_rank_list[i][p],Second_rank_list[i][p],Third_rank_list[i][p])
        toptwo_rank_mapping.append(toptwo)
        topthree_rank_mapping.append(topthree)
        
        
    topfirstrank_weig_list.append(first_rank_mapping)
    toptworank_weig_list.append(toptwo_rank_mapping)
    topthreerank_weig_list.append(topthree_rank_mapping)


   

##### Accuracy calculation #####

In [246]:
top_firstrank_accuracy = [] 
top_tworank_accuracy =  [] 
top_threerank_accuracy = [] 

for a in range(len(topfirstrank_weig_list)):
    
    topone_matching = sum([sum(l) for l in topfirstrank_weig_list[a]])/ len(totalsen_no)
    toptwo_matching = sum([sum(l) for l in toptworank_weig_list[a]])/ len(totalsen_no)
    topthree_matching = sum([sum(l) for l in topthreerank_weig_list[a]])/ len(totalsen_no)
    
    top_firstrank_accuracy.append(topone_matching)
    top_tworank_accuracy.append(toptwo_matching)
    top_threerank_accuracy.append(topthree_matching)


In [249]:
print(weights)
print(top_firstrank_accuracy)
print(top_tworank_accuracy) 
print(top_threerank_accuracy)

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
[0.3721275357024624, 0.4289660840877179, 0.47354421854130757, 0.5200401831071131, 0.5534195595840136, 0.5753604493201977, 0.5896300186075184, 0.5974954051986895, 0.6014908845991392]
[0.6096987408532061, 0.6677701800248861, 0.6998710030936426, 0.7269146908069727, 0.748296213427094, 0.7629082523773102, 0.7717325540245893, 0.7771321590429114, 0.7802828799415519]
[0.7717782166463087, 0.8117101793399468, 0.8306373360426489, 0.847737987876574, 0.8593705407595977, 0.8681149328188678, 0.8735145378371899, 0.8771104692975947, 0.8789369741663717]


##### Weight - Accuracy Plot #####

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go


In [248]:
trace0 = go.Scatter(
    x = weights,
    y = topfirstrank_accuracy,
    name = 'Actualanswer sentence = FirstRank ',
    line = dict(
        
        width = 4,dash='dot')
)
trace1 = go.Scatter(
    x = weights,
    y = top_tworank_accuracy,
    name = 'Actualanswer sentence in Toptwo rank',
    mode = 'lines',

    line = dict(

        width = 2,)
)

trace2 = go.Scatter(
    x = weights,
    y = top_threerank_accuracy,
    name = 'Actualanswer sentence in Topthree rank',
    mode = 'lines',

    line = dict(

        width = 2,)
)


data = [trace0, trace1,trace2]

# Edit the layout
layout = dict(
              xaxis = dict(title = 'Weights'),
              yaxis = dict(title = 'Accuracy'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [257]:
df = pd.DataFrame(weights,columns = ['Weights'])
df['Topone_accuracy'] = top_firstrank_accuracy
df['Toptwo_accuracy'] = top_tworank_accuracy
df['Topthree_accuracy'] = top_threerank_accuracy
df

,Weights,Topone_accuracy,Toptwo_accuracy,Topthree_accuracy
0,0.1,0.372128,0.609699,0.771778
1,0.2,0.428966,0.667770,0.811710
2,0.3,0.473544,0.699871,0.830637
3,0.4,0.520040,0.726915,0.847738
4,0.5,0.553420,0.748296,0.859371
5,0.6,0.575360,0.762908,0.868115
6,0.7,0.589630,0.771733,0.873515
7,0.8,0.597495,0.777132,0.877110
8,0.9,0.601491,0.780283,0.878937


############################################################################################################

In [567]:
egsn = [] 
egans = [] 
paragraph_pointer = sqd_data.data[0]['paragraphs'] 

for b in range(len(paragraph_pointer)):
    processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[b]['context']) 
    que_list = preprocess_question(paragraph_pointer[b]['qas'])
    processed_ans_list = answerprocessfun(paragraph_pointer[b])
    egsn.append(process_sen_tokenized_list)
    egans.append(processed_ans_list)


In [620]:
len(egsn[0])

7

In [571]:
egans[0]

[['saint', 'bernadette', 'soubirous'],
 ['a', 'copper', 'statue', 'of', 'christ'],
 ['the', 'main', 'building'],
 ['a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection'],
 ['a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary']]

##### Actual answer list, Candidate answer list #####

In [735]:
def answerprocessfun(context):
    answer_ls = [] 
    for i in range(len(context['qas'])):
       
        answer_ls.append(context['qas'][i]['answers'][0]['text'].lower().split())
    return answer_ls

In [563]:
len(sqd_data)                      

442

In [762]:
sqd_data.data[0]['paragraphs'] 


[{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'qas': [{'answers': [{'answer_start': 515,
      'text': 'Saint Bernadette Soubirous'}],
    'id': '5733be284776f41900661182',
    'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
   {'answers': [{'answer_start': 188, 'text': 'a copper statue of 

In [564]:
qn_sen_rank_list = [] 
actualans_list = []
candans1_list = []
candans2_list = [] 

for a in range(len(sqd_data)):
    #Content_list.append(b)                                 
    
    #Qn_list = [] 
    paragraph_pointer = sqd_data.data[a]['paragraphs'] 
    

    for b in range(len(paragraph_pointer)):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[b]['context']) 
        que_list = preprocess_question(paragraph_pointer[b]['qas']) 
        processed_ans_list = answerprocessfun(paragraph_pointer[b])
        comm_word = [] 
        
        for c in range(len(processed_ans_list)):
            for d in range(len(process_sen_tokenized_list)):
                ls = len(set(processed_ans_list[c]) & set(process_sen_tokenized_list[d][0]))
                comm_word.append([c,d,ls]) 
                qnsen_df = pd.DataFrame(comm_word,columns=['Qnno','Sentno','#comm_word'])
                qnsen_df['Rank'] = qnsen_df.groupby('Qnno')['#comm_word'].rank(method = "first",ascending = False)
                actualans = list(qnsen_df[qnsen_df['Rank']==1.0]['Sentno'])
                candans1 = list(qnsen_df[qnsen_df['Rank']==2.0]['Sentno'])
                candans2 = list(qnsen_df[qnsen_df['Rank']==3.0]['Sentno']) 
        qn_sen_rank_list.append(qnsen_df) 
        actualans_list.append(actualans)
        candans1_list.append(candans1)
        candans2_list.append(candans2)
        


In [565]:
len(qn_sen_rank_list)

18896

* We have 18896 paragrapgh.

In [566]:
qn_sen_rank_list[0]

,Qnno,Sentno,#comm_word,Rank
0,0,0,0,2.0
1,0,1,0,3.0
2,0,2,0,4.0
3,0,3,0,5.0
4,0,4,0,6.0
5,0,5,3,1.0
6,0,6,0,7.0
7,1,0,0,4.0
8,1,1,1,2.0
9,1,2,3,1.0


##### W2V distance ##### 

In [594]:
processed_que_list

['Which national event caused Beyonce to produce "Demand a Plan?"',
 'What song did Beyonce contribute to the campaign?',
 'Beyonce is contributing to which food-donation campaign?',
 'On June 1 , 2013 where was the concert held for "a Chime for Change"?',
 'Beyonce was speaking about whom when she said her gift was "finding the best qualities in every human being."?',
 'The Demand a Plan video campaign followed what tragic event?',
 'What was the focus of the Gucci Chime for Change campaign?',
 'What crowdfunding platform was used in the concert?',
 'What is the name of the campaign that Beyonce and others are involved in that deals with gun control?',
 'What school shooting prompted the creation of Demand A Plan?',
 'What song did Beyonce donate to the 2012 World Humanitarian Day campaign?',
 'Who did Beyonce work with in 2013 on the Chime for Change campaign?']

In [604]:
processed_sen_list

['Notre Dame alumni work in various fields.',
 'Alumni working in political fields include state governors, members of the United States Congress, and former United States Secretary of State Condoleezza Rice.',
 'A notable alumnus of the College of Science is Medicine Nobel Prize winner Eric F. Wieschaus.',
 "A number of university heads are alumni, including Notre Dame's current president, the Rev.",
 'John Jenkins.',
 'Additionally, many alumni are in the media, including talk show hosts Regis Philbin and Phil Donahue, and television and radio personalities such as Mike Golic and Hannah Storm.',
 'With the university having high profile sports teams itself, a number of alumni went on to become involved in athletics outside the university, including professional baseball, basketball, football, and ice hockey players, such as Joe Theismann, Joe Montana, Tim Brown, Ross Browner, Rocket Ismail, Ruth Riley, Jeff Samardzija, Jerome Bettis, Brett Lebda, Olympic gold medalist Mariel Zagunis,

##### W2V, Tag distance #####

In [120]:
def w2vdistancefun(que_list,sen_list):
    distance = [] 
    for m in range(len(que_list)):
        for j in range(len(sen_list)):

                    #word2vec_model.init_sims(replace=True)
            score = word2vec_model.wmdistance(que_list[m].split(), sen_list[j].split())
            score_df = [m,j,score]
            distance.append(score_df)
            distance_df = pd.DataFrame(distance,columns = ['Qnno','Sentno','w2vDistance'])
            distance_df['Rank'] = distance_df.groupby('Qnno')['w2vDistance'].rank(method = "first",ascending=True)
            anssent = list(distance_df[distance_df['Rank']==1.0]['Sentno']) 
    return distance_df,anssent
                

In [121]:
def tagdistancefun(pro_qnlist,pro_senlist):
    ls = [] 

    que_type_clean = qntypefun(pro_qnlist)

    for m in range(len(que_type_clean)):

        map_que_ner = []
        map_que_ner.append(que_type_clean[m])
        for j in range(len(pro_senlist)):



            map_ans_ner = []
            ans_ner = nlp(pro_senlist[j])
            for ent in ans_ner.ents:
                map_ans_ner.append('a_' + ent.label_)
            unique_ner = ner_combination(map_que_ner, map_ans_ner)
                
            d_1 = bag_of_words(map_que_ner, unique_ner)

            d_2 = bag_of_words(map_ans_ner, unique_ner)
            sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
            sub_matrix= sub_matrix_df.as_matrix()
            sub_matrix[np.isnan(sub_matrix)] = 0

            score = emd(d_1, d_2, sub_matrix)
               
            ls.append([m,j,score]) 
                
                
                
            df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
            df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

            anssent = list(df[df['Rank']==1.0]['Sentno'])
    return df,anssent
            
            
    

In [122]:
def tagdistancefuner(pro_qnlist,pro_senlist):
    ls = [] 

    que_type_clean = qntypefun(pro_qnlist)
    for i in range(len(pro_qnlist)):
        query = pro_qnlist[i]


        #for m in range(len(que_type_clean)):

            #map_que_ner = []
            #map_que_ner.append(que_type_clean[m])

    for m in range(len(que_type_clean)):

        map_que_ner = []
        que_ner = nlp(query)
        for ent in que_ner.ents:
            map_que_ner.append('q_' + ent.label_)
        map_que_ner.append(que_type_clean[m])

        for j in range(len(pro_senlist)):



            map_ans_ner = []
            ans_ner = nlp(pro_senlist[j])
            for ent in ans_ner.ents:
                map_ans_ner.append('a_' + ent.label_)
            unique_ner = ner_combination(map_que_ner, map_ans_ner)

            d_1 = bag_of_words(map_que_ner, unique_ner)

            d_2 = bag_of_words(map_ans_ner, unique_ner)
            sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
            sub_matrix= sub_matrix_df.as_matrix()
            sub_matrix[np.isnan(sub_matrix)] = 0

            score = emd(d_1, d_2, sub_matrix)

            ls.append([m,j,score]) 



            df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])

            df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

            anssent = list(df[df['Rank']==1.0]['Sentno'])
    
    return df,anssent
            
            
    

In [775]:
tag_ans_sent[0]

[1, 0, 0, 0, 0]

In [666]:
w2vdistance = [] 
w2v_ans_sent = [] 
tagdistance = [] 
tag_ans_sent = [] 
tagdistance_ner = []
tagdistance_ans_ner = [] 

for i in range(len(sqd_data)):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 
    

    for k in range(len(paragraph_pointer)):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])
        w2v,w2vans = w2vdistancefun(processed_que_list,processed_sen_list) 
        tag, tagans = tagdistancefun(processed_que_list,processed_sen_list)
        tagner,tagnerans = tagdistancefuner(processed_que_list,processed_sen_list)
        
        w2vdistance.append(w2v)
        w2v_ans_sent.append(w2vans) 
        tagdistance.append(tag)
        tag_ans_sent.append(tagans)
        tagdistance_ner.append(tagner)
        tagdistance_ans_ner.append(tagnerans)
        

In [790]:
tagdistancefuner(processed_que_list,processed_sen_list)


(    Qnno  Sentno  TagDistance  Rank
 0      0       0     0.344255  10.0
 1      0       1     0.312768   5.0
 2      0       2     0.278807   1.0
 3      0       3     0.303877   4.0
 4      0       4     0.326570   9.0
 5      0       5     0.286905   2.0
 6      0       6     0.325806   7.0
 7      0       7     0.314687   6.0
 8      0       8     0.325835   8.0
 9      0       9     0.302462   3.0
 10     0      10     0.399680  11.0
 11     1       0     0.353198   9.0
 12     1       1     0.362439  10.0
 13     1       2     0.302591   2.0
 14     1       3     0.312820   3.0
 15     1       4     0.346491   7.0
 16     1       5     0.296979   1.0
 17     1       6     0.345727   6.0
 18     1       7     0.334608   5.0
 19     1       8     0.352748   8.0
 20     1       9     0.324940   4.0
 21     1      10     0.411058  11.0
 22     2       0     0.376948  11.0
 23     2       1     0.356215   9.0
 24     2       2     0.288702   1.0
 25     2       3     0.301043   2.0
 

In [668]:
print(len(w2vdistance))
print(len(w2v_ans_sent))
print(len(tagdistance))
print(len(tag_ans_sent))

18896
18896
18896
18896


In [763]:
ls = [] 
ls1 = [] 
paragraph_pointer = sqd_data.data[0]['paragraphs'] 
    

for k in range(len(paragraph_pointer)):
    processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
    processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])
    ls.append(processed_sen_list)
    ls1.append(processed_que_list)

In [765]:
ls[0]

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [767]:
ls1[0] 

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?']

In [671]:
tagdistance[0]

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.378392,4.0
1,0,1,0.349630,1.0
2,0,2,0.416877,7.0
3,0,3,0.378679,5.0
4,0,4,0.371880,3.0
5,0,5,0.391814,6.0
6,0,6,0.350135,2.0
7,1,0,0.397277,1.0
8,1,1,0.450303,5.0
9,1,2,0.482885,7.0


In [759]:
tag_ans_sent[0]

[1, 0, 0, 0, 0]

In [758]:
w2vdistance[0]

,Qnno,Sentno,w2vDistance,Rank
0,0,0,2.652292,3.0
1,0,1,2.631091,2.0
2,0,2,2.906310,7.0
3,0,3,2.734407,5.0
4,0,4,2.763077,6.0
5,0,5,2.120559,1.0
6,0,6,2.690732,4.0
7,1,0,2.678404,7.0
8,1,1,2.437435,3.0
9,1,2,2.435459,2.0


In [760]:
w2v_ans_sent[0]

[5, 3, 3, 3, 3]

###### Weighted distance  ######

In [761]:
weights

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [672]:
First_rank_list = [] 
Second_rank_list = []
Third_rank_list = [] 
Distance_list = [] 

for i in range(len(w2vdistance)):
    first_rank = [] 
    second_rank = [] 
    third_rank = [] 
    weighted_distance_list = [] 

    for a in range(len(weights)):

        weigDistance = pd.DataFrame((w2vdistance[i]['w2vDistance']*weights[a]) + (tagdistance[i]['TagDistance']*(1-weights[a])),columns = ['WeigDistance']) 
        weigDistance['Qnno'] = w2vdistance[i]['Qnno'] 
        weigDistance['Sentno'] =  w2vdistance[i]['Sentno']
        weigDistance['WeiRank'] = weigDistance.groupby('Qnno')['WeigDistance'].rank(method = "first",ascending=True)
        first_rank.append(list(weigDistance[weigDistance['WeiRank']==1.0]['Sentno']))
        second_rank.append(list(weigDistance[weigDistance['WeiRank']==2.0]['Sentno']))
        third_rank.append(list(weigDistance[weigDistance['WeiRank']==3.0]['Sentno'])) 
        weighted_distance_list.append(weigDistance)


        
    First_rank_list.append(first_rank) 
    Second_rank_list.append(second_rank)
    Third_rank_list.append(third_rank)
    Distance_list.append(weighted_distance_list) 
    

##### Zero- One mapping #####

In [123]:
def toponemapping(list1,list2):
    ls = []
    for i in range(len(list1)):
        if (list1[i]==list2[i]): 
            ls.append(1) 
        else:
            ls.append(0)
            
    return ls 

In [124]:
def topthreemapping(list1,list2,list3,list4):
    ls = []
    for i in range(len(list2)):
        if (list1[i]==list2[i]) or (list1[i]==list3[i]) or (list1[i]==list4[i]): 
            ls.append(1) 
        else:
            ls.append(0)
            
    return ls 

In [675]:
weights

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [676]:
weipredi_actual_mapping_list = []  
weipredi_candidate_mapping_list = [] 

for p in range(len(weights)):

    first_rank_mapping = []
    topthree_rank_mapping = []  
    for i in range(len(actualans_list)):

        first_rank_mapping.append(toponemapping(actual_anssent[i],First_rank_list[i][p]))
        topthree_rank_mapping.append(topthreemapping(First_rank_list[i][p],actual_anssent[i],candans1_list[i],candans2_list[i])) 
        
        
    weipredi_actual_mapping_list.append(first_rank_mapping)
    weipredi_candidate_mapping_list.append(topthree_rank_mapping)


In [686]:
w2v_mapping = [] 
tag_mapping = [] 
for i in range(len(w2v_ans_sent)):
    w2v_mapping.append(toponemapping(actual_anssent[i],w2v_ans_sent[i]))
    tag_mapping.append(toponemapping(actual_anssent[i],tag_ans_sent[i]))
    
               

In [757]:
len(weipredi_actual_mapping_list[0])

18896

###### Accuracy ######

In [701]:
w2v_accuracy = sum([sum(l) for l in w2v_mapping])/ sum([len(l) for l in actualans_list])
tag_accuracy = sum([sum(l) for l in tag_mapping])/ sum([len(l) for l in actualans_list])
   

In [702]:
w2v_accuracy

0.6042763045240243

In [703]:
tag_accuracy

0.22247970867247344

In [692]:
predict_actual_accuracy = [] 
predict_candidate_accuracy = [] 



for a in range(len(topfirstrank1_weig_list)):
    topone_matching = sum([sum(l) for l in topfirstrank_weig_list[a]])/ sum([len(l) for l in actualans_list])
    candidate_matching = sum([sum(l) for l in weipredi_candidate_mapping_list[a]])/ sum([len(l) for l in actualans_list])

    predict_actual_accuracy.append(topone_matching)
    predict_candidate_accuracy.append(candidate_matching)
    

In [740]:
predict_actual_accuracy_df = pd.DataFrame(weights, columns = ['Weights'])
predict_actual_accuracy_df['Accuracy'] = predict_actual_accuracy
predict_actual_accuracy_df


,Weights,Accuracy
0,0.1,0.003961
1,0.2,0.004783
2,0.3,0.005491
3,0.4,0.006096
4,0.5,0.006324
5,0.6,0.006530
6,0.7,0.006587
7,0.8,0.006690
8,0.9,0.006724


In [741]:
#sum([len(l) for l in actualans_list])

In [739]:
predict_candidate_accuracy_df = pd.DataFrame(weights, columns = ['Weights'])
predict_candidate_accuracy_df['Accuracy'] = predict_candidate_accuracy
predict_candidate_accuracy_df

,Weights,Accuracy
0,0.1,0.707988
1,0.2,0.737497
2,0.3,0.765283
3,0.4,0.794153
4,0.5,0.815797
5,0.6,0.829450
6,0.7,0.838240
7,0.8,0.843594
8,0.9,0.845934


######################################################################################################################

In [706]:
print(len(w2v_mapping))
print(len(tag_mapping))

18896
18896


In [711]:
weights

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [734]:
trace0 = go.Scatter(
    x = weights,
    y = predict_actual_accuracy,
    name = 'TopRank weighted accuracy',
    line = dict(
        
        width = 4,dash='dot')
)
trace1 = go.Scatter(
    x = weights,
    y = predict_candidate_accuracy,
    name = 'TopthreeRank weighted accuracy',
    mode = 'lines',

    line = dict(

        width = 2,)
)



data = [trace0, trace1]

# Edit the layout
layout = dict(
              xaxis = dict(title = 'Weights'),
              yaxis = dict(title = 'Accuracy'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig)

###################################################################################################################### 

In [125]:
w2vdistance = [] 
w2v_ans_sent = [] 
tagdistance = [] 
tag_ans_sent = [] 
tagdistance_ner = []
tagdistance_ans_ner = [] 

for i in range(0,1):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 
    

    for k in range(0,2):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])
        w2v,w2vans = w2vdistancefun(processed_que_list,processed_sen_list) 
        tag, tagans = tagdistancefun(processed_que_list,processed_sen_list)
        tagner,tagnerans = tagdistancefuner(processed_que_list,processed_sen_list)
        
        w2vdistance.append(w2v)
        w2v_ans_sent.append(w2vans) 
        tagdistance.append(tag)
        tag_ans_sent.append(tagans)
        tagdistance_ner.append(tagner)
        tagdistance_ans_ner.append(tagnerans)
        

In [149]:
def w2vdistancefun1(que_list,sen_list):
    distance = [] 
    for m in range(len(que_list)):
        for j in range(len(sen_list)):

                    #word2vec_model.init_sims(replace=True)
            score = word2vec_model.wmdistance(que_list[m].split(), sen_list[j].split())
            score_df = [m,j,score] 
            distance.append(score_df)
            distance_df = pd.DataFrame(distance,columns = ['Qnno','Sentno','w2vDistance'])
            uniqa = distance_df['Qnno'].unique() 
#div=[w2vdistance[0]['w2vDistance'][w2vdistance[0]['Qnno']==i]/max(w2vdistance[0]['w2vDistance'][w2vdistance[0]['Qnno']==i]) for i in uniqa]


            div= [distance_df['w2vDistance'][distance_df['Qnno']==i]/max(distance_df['w2vDistance'][distance_df['Qnno']==i]) for i in uniqa]
            flat_list = [item for sublist in div for item in sublist]
            distance_df['w2vDistance']= flat_list
            
            distance_df['Rank'] = distance_df.groupby('Qnno')['w2vDistance'].rank(method = "first",ascending=True)
            anssent = list(distance_df[distance_df['Rank']==1.0]['Sentno']) 
    return distance_df,anssent
                
            

In [174]:
def w2vdistancefun11(que_list,sen_list):
    distance = [] 
    for m in range(len(que_list)):
        for j in range(len(sen_list)):

                    #word2vec_model.init_sims(replace=True)
            score = word2vec_model.wmdistance(que_list[m].split(), sen_list[j].split())
            score_df = [m,j,score] 
            distance.append(score_df)
            distance_df = pd.DataFrame(distance,columns = ['Qnno','Sentno','w2vDistance'])
            #uniqa = distance_df['Qnno'].unique() 


            #div= [distance_df['w2vDistance'][distance_df['Qnno']==i]/max(distance_df['w2vDistance'][distance_df['Qnno']==i]) for i in uniqa]
            #flat_list = [item for sublist in div for item in sublist]
            distance_df['new']= distance_df['w2vDistance']/max( distance_df['w2vDistance'])
            
            distance_df['Rank'] = distance_df.groupby('Qnno')['new'].rank(method = "first",ascending=True)
            anssent = list(distance_df[distance_df['Rank']==1.0]['Sentno']) 
    return distance_df,anssent
                
            

In [162]:
paragraph_pointer = sqd_data.data[0]['paragraphs'] 
processed_sen_list, process_sen_token_list = preprocess_text(paragraph_pointer[2]['context'])
print(processed_sen_list)
processed_que_list = preprocess_question(paragraph_pointer[2]['qas'])
print()
print(processed_que_list)


['The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome).', 'Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building.', 'Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians.', 'Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto.', 'The university through the Moreau Seminary has ties to theologian Frederick Buechner.', 'While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.']

['Where is the headquarters of the Congregation of the Holy Cross?', 'What is the primary seminary of the Congregation of the Holy Cross?', 'What is the oldest structure at Notre Dame?', 'What individuals live at Fatima House at Notre Dame?', 'Which prize did Frederick Buech

In [185]:
1.849660/3.5321449298684957


0.5236648089830404

In [184]:
max(l2['w2vDistance'])

3.5321449298684957

In [183]:
l2 = w2vdistancefun(processed_que_list,processed_sen_list)[0]
l2['new'] = l2['w2vDistance']/max(l2['w2vDistance'])
l2['new_rank'] = l2.groupby('Qnno')['new'].rank(method = "first",ascending=True)
l2

,Qnno,Sentno,w2vDistance,Rank,new,new_rank
0,0,0,1.849660,1.0,0.523665,1.0
1,0,1,2.520986,2.0,0.713727,2.0
2,0,2,2.773693,4.0,0.785272,4.0
3,0,3,2.746212,3.0,0.777491,3.0
4,0,4,2.891435,5.0,0.818606,5.0
5,0,5,3.261910,6.0,0.923493,6.0
6,1,0,1.823674,1.0,0.516308,1.0
7,1,1,2.531645,2.0,0.716744,2.0
8,1,2,2.802277,5.0,0.793364,5.0
9,1,3,2.763377,3.0,0.782351,3.0


In [186]:
w2vdistancefun1(processed_que_list,processed_sen_list)[0]

,Qnno,Sentno,w2vDistance,Rank
0,0,0,0.567048,1.0
1,0,1,0.772856,2.0
2,0,2,0.850328,4.0
3,0,3,0.841903,3.0
4,0,4,0.886424,5.0
5,0,5,1.000000,6.0
6,1,0,0.571454,1.0
7,1,1,0.793298,2.0
8,1,2,0.878101,5.0
9,1,3,0.865912,3.0


In [175]:
w2vdistancefun11(processed_que_list,processed_sen_list)[0].tail()

,Qnno,Sentno,w2vDistance,new,Rank
25,4,1,3.258495,0.922526,4.0
26,4,2,3.392508,0.960467,5.0
27,4,3,3.532145,1.000000,6.0
28,4,4,3.136071,0.887866,2.0
29,4,5,2.883665,0.816406,1.0


In [137]:
w2vdistance[1]

,Qnno,Sentno,w2vDistance,Rank
0,0,0,2.880365,2.0
1,0,1,3.305640,8.0
2,0,2,2.849773,1.0
3,0,3,3.311028,9.0
4,0,4,3.538828,11.0
5,0,5,3.096189,5.0
6,0,6,2.961657,3.0
7,0,7,3.235958,7.0
8,0,8,3.091528,4.0
9,0,9,3.223291,6.0


In [129]:
w2vdistance[0]
uniqa = w2vdistance[0]['Qnno'].unique() 

div=[w2vdistance[0]['w2vDistance'][w2vdistance[0]['Qnno']==i]/max(w2vdistance[0]['w2vDistance'][w2vdistance[0]['Qnno']==i]) for i in uniqa]
flat_list = [item for sublist in div for item in sublist]
w2vdistance[0]['c']= flat_list

In [132]:
2.652292/2.906310

0.9125977614225599

In [130]:
w2vdistance[0]


,Qnno,Sentno,w2vDistance,Rank,c
0,0,0,2.652292,3.0,0.912598
1,0,1,2.631091,2.0,0.905303
2,0,2,2.906310,7.0,1.000000
3,0,3,2.734407,5.0,0.940852
4,0,4,2.763077,6.0,0.950717
5,0,5,2.120559,1.0,0.729640
6,0,6,2.690732,4.0,0.925824
7,1,0,2.678404,7.0,1.000000
8,1,1,2.437435,3.0,0.910033
9,1,2,2.435459,2.0,0.909295


In [213]:
df = pd.DataFrame([[0,2],[0,6],[1,2],[1,1],[2,60],[2,23]],columns = ['a','b'])
df['RANk'] = df.groupby('a')['b'].rank(method = "first",ascending = True)

In [214]:
uniqa =df['a'].unique() 

div=[df['b'][df['a']==i]/max(df['b'][df['a']==i]) for i in uniqa]
type(div)
#df[df['a']==i,:].value/max(df['b'][df['a']==i])

#df[df['a']==i,:]/max(df['b'][df['a']==i]) for i in uniqa

list

In [215]:


flat_list = [item for sublist in div for item in sublist]
df['c']= flat_list
df['rank'] = df.groupby('a')['c'].rank(method = "first",ascending=True)
df['d']= df['b']/df['b'].max() 
df['Rank'] = df.groupby('a')['d'].rank(method = "first",ascending = True)
df

,a,b,RANk,c,rank,d,Rank
0,0,2,1.0,0.333333,1.0,0.033333,1.0
1,0,6,2.0,1.000000,2.0,0.100000,2.0
2,1,2,2.0,1.000000,2.0,0.033333,2.0
3,1,1,1.0,0.500000,1.0,0.016667,1.0
4,2,60,2.0,1.000000,2.0,1.000000,2.0
5,2,23,1.0,0.383333,1.0,0.383333,1.0


In [201]:
df['b']/df['b'].max()

0    0.333333
1    1.000000
2    0.166667
3    0.333333
Name: b, dtype: float64

In [74]:
df

,a,b
0,0,2
1,0,6
2,1,1
3,1,2


In [56]:
for i in list(df['a'].unique()):

    for j in df['b']:
        print(j/max(df['b']))


0.333333333333
1.0
0.166666666667
0.333333333333
0.333333333333
1.0
0.166666666667
0.333333333333


In [52]:
for i in list(df['a'].unique()):
    for j in df['b']:
        df['c'] = j/max(j)


TypeError: 'numpy.int64' object is not iterable

In [49]:
df

,a,b,c
0,1,2,0.333333
1,1,6,1.000000
2,0,1,0.166667
3,0,2,0.333333


In [801]:
w2vdistancefun(processed_que_list,processed_sen_list) 


(    Qnno  Sentno  w2vDistance  Rank
 0      0       0     2.880365   2.0
 1      0       1     3.305640   8.0
 2      0       2     2.849773   1.0
 3      0       3     3.311028   9.0
 4      0       4     3.538828  11.0
 5      0       5     3.096189   5.0
 6      0       6     2.961657   3.0
 7      0       7     3.235958   7.0
 8      0       8     3.091528   4.0
 9      0       9     3.223291   6.0
 10     0      10     3.319965  10.0
 11     1       0     2.663530   2.0
 12     1       1     3.188587  11.0
 13     1       2     2.851131   4.0
 14     1       3     2.795269   3.0
 15     1       4     3.083074   8.0
 16     1       5     2.972072   7.0
 17     1       6     2.968117   6.0
 18     1       7     3.110212   9.0
 19     1       8     2.881502   5.0
 20     1       9     2.568720   1.0
 21     1      10     3.133448  10.0
 22     2       0     2.183062   1.0
 23     2       1     3.098949  11.0
 24     2       2     2.690508   6.0
 25     2       3     2.371992   3.0
 

In [792]:
tagdistance[0].head()

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.378392,4.0
1,0,1,0.349630,1.0
2,0,2,0.416877,7.0
3,0,3,0.378679,5.0
4,0,4,0.371880,3.0


In [793]:
tagdistance_ner[0].head()

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.446360,7.0
1,0,1,0.361591,1.0
2,0,2,0.373903,3.0
3,0,3,0.371100,2.0
4,0,4,0.386076,4.0


In [794]:
paragraph_pointer = sqd_data.data[0]['paragraphs'] 
processed_sen_list, process_sen_token_list = preprocess_text(paragraph_pointer[0]['context'])
processed_que_list = preprocess_question(paragraph_pointer[0]['qas'])


In [799]:
agdistance = [] 
ag_ans_sent = [] 

for i in range(0,1):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    for k in range(0,2):#no of paragraphs in a row of the df

        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])

        

        ls = [] 

        que_type_clean = qntypefun(processed_que_list)
        for i in range(len(processed_que_list)):
            query = processed_que_list[i]


            #for m in range(len(que_type_clean)):

                #map_que_ner = []
                #map_que_ner.append(que_type_clean[m])

        for m in range(len(que_type_clean)):

            map_que_ner = []
            que_ner = nlp(query)
            for ent in que_ner.ents:
                map_que_ner.append('q_' + ent.label_)
            map_que_ner.append(que_type_clean[m])

            for j in range(len(processed_sen_list)):



                map_ans_ner = []
                ans_ner = nlp(processed_sen_list[j])
                for ent in ans_ner.ents:
                    map_ans_ner.append('a_' + ent.label_)
                #print("Ans_NER", " ", map_ans_ner)
                unique_ner = ner_combination(map_que_ner, map_ans_ner)
                #print("Unique_NER", " ", unique_ner)
                #print()
                distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
                d_1 = bag_of_words(map_que_ner, unique_ner)
                #print(d_1)

                d_2 = bag_of_words(map_ans_ner, unique_ner)
                #print(d_2)
                sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
                #print(sub_matrix_df)
                sub_matrix= sub_matrix_df.as_matrix()
                sub_matrix[np.isnan(sub_matrix)] = 0

                #print(sub_matrix)
                score = emd(d_1, d_2, sub_matrix)
                ls.append([m,j,score]) 

                df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
                df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

        agdistance.append(df)
        ag_ans_sent.append(list(df[df['Rank']==1.0]['Sentno'])) 
    
               

In [800]:
agdistance[0].head()

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.446360,7.0
1,0,1,0.361591,1.0
2,0,2,0.373903,3.0
3,0,3,0.371100,2.0
4,0,4,0.386076,4.0


In [653]:
Tagdistance = [] 
Tag_ans_sent = [] 

for i in range(0,2):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    for k in range(len(paragraph_pointer)):#no of paragraphs in a row of the df

        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])

        

        ls = [] 

        que_type_clean = qntypefun(processed_que_list)

        for m in range(len(que_type_clean)):

            map_que_ner = []
            map_que_ner.append(que_type_clean[m])
            #print("Que_NER", " ", map_que_ner)
            for j in range(len(processed_sen_list)):



                map_ans_ner = []
                ans_ner = nlp(processed_sen_list[j])
                for ent in ans_ner.ents:
                    map_ans_ner.append('a_' + ent.label_)
                #print("Ans_NER", " ", map_ans_ner)
                unique_ner = ner_combination(map_que_ner, map_ans_ner)
                #print("Unique_NER", " ", unique_ner)
                #print()
                # distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
                d_1 = bag_of_words(map_que_ner, unique_ner)
                #print(d_1)

                d_2 = bag_of_words(map_ans_ner, unique_ner)
                #print(d_2)
                sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
                #print(sub_matrix_df)
                sub_matrix= sub_matrix_df.as_matrix()
                sub_matrix[np.isnan(sub_matrix)] = 0

                #print(sub_matrix)
                score = emd(d_1, d_2, sub_matrix)
                #print(processed_sen_list[j])
                #print('Score = ', score)
                #l1 = [m,j,score] 
                #ls.append(pd.DataFrame([m,j,score])
                #tagdis = pd.DataFrame(l1) 
                #tagdis['TagRank'] = tagdis.groupby('Qnno')['Tagdis'].rank(method = "first",ascending=True)
                #Tagdistance.append(ls)
                ls.append([m,j,score]) 
                
                
                
                df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
                df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

        Tagdistance.append(df)
        Tag_ans_sent.append(list(df[df['Rank']==1.0]['Sentno'])) 
    
        


In [616]:
w2vdistance[0]

,Qnno,Sentno,w2vDistance,Rank
0,0,0,2.868377,1.0
1,0,1,3.145677,3.0
2,0,2,3.800246,7.0
3,0,3,3.058285,2.0
4,0,4,3.542142,4.0
5,0,5,3.608783,5.0
6,0,6,3.659406,6.0
7,0,7,3.869648,8.0
8,1,0,2.533910,2.0
9,1,1,3.387028,6.0


In [608]:
len(w2vdistance)

55

In [612]:
w2vdistance

[    Qnno  Sentno  w2vDistance  Rank
 0      0       0     2.652292   3.0
 1      0       1     2.631091   2.0
 2      0       2     2.906310   7.0
 3      0       3     2.734407   5.0
 4      0       4     2.763077   6.0
 5      0       5     2.120559   1.0
 6      0       6     2.690732   4.0
 7      1       0     2.678404   7.0
 8      1       1     2.437435   3.0
 9      1       2     2.435459   2.0
 10     1       3     2.067550   1.0
 11     1       4     2.542102   6.0
 12     1       5     2.465135   4.0
 13     1       6     2.470335   5.0
 14     2       0     2.822904   5.0
 15     2       1     2.855032   6.0
 16     2       2     2.868838   7.0
 17     2       3     1.851679   1.0
 18     2       4     2.502111   2.0
 19     2       5     2.519347   3.0
 20     2       6     2.680416   4.0
 21     3       0     2.618026   5.0
 22     3       1     2.740448   6.0
 23     3       2     2.880111   7.0
 24     3       3     2.373554   1.0
 25     3       4     2.481385   3.0
 

In [586]:
(w2vdistance)

<function __main__.w2vdistance>

###### Tag distance ##### 

In [411]:
Tagdistance = [] 
Tag_ans_sent = [] 

for i in range(0,2):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    for k in range(len(paragraph_pointer)):#no of paragraphs in a row of the df

        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])

        

        ls = [] 

        que_type_clean = qntypefun(processed_que_list)

        for m in range(len(que_type_clean)):

            map_que_ner = []
            map_que_ner.append(que_type_clean[m])
            #print("Que_NER", " ", map_que_ner)
            for j in range(len(processed_sen_list)):



                map_ans_ner = []
                ans_ner = nlp(processed_sen_list[j])
                for ent in ans_ner.ents:
                    map_ans_ner.append('a_' + ent.label_)
                #print("Ans_NER", " ", map_ans_ner)
                unique_ner = ner_combination(map_que_ner, map_ans_ner)
                #print("Unique_NER", " ", unique_ner)
                #print()
                # distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
                d_1 = bag_of_words(map_que_ner, unique_ner)
                #print(d_1)

                d_2 = bag_of_words(map_ans_ner, unique_ner)
                #print(d_2)
                sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
                #print(sub_matrix_df)
                sub_matrix= sub_matrix_df.as_matrix()
                sub_matrix[np.isnan(sub_matrix)] = 0

                #print(sub_matrix)
                score = emd(d_1, d_2, sub_matrix)
                #print(processed_sen_list[j])
                #print('Score = ', score)
                #l1 = [m,j,score] 
                #ls.append(pd.DataFrame([m,j,score])
                #tagdis = pd.DataFrame(l1) 
                #tagdis['TagRank'] = tagdis.groupby('Qnno')['Tagdis'].rank(method = "first",ascending=True)
                #Tagdistance.append(ls)
                ls.append([m,j,score]) 
                
                
                
                df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
                df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

        Tagdistance.append(df)
        Tag_ans_sent.append(list(df[df['Rank']==1.0]['Sentno'])) 
    

In [634]:
tagdistancefun(processed_que_list,processed_sen_list)

(    Qnno  Sentno  TagDistance  Rank
 0      0       0     0.456953   7.0
 1      0       1     0.352180   6.0
 2      0       2     0.329682   5.0
 3      0       3     0.456953   8.0
 4      0       4     0.230314   1.0
 5      0       5     0.230314   2.0
 6      0       6     0.232325   4.0
 7      0       7     0.230314   3.0
 8      1       0     0.456953   7.0
 9      1       1     0.352180   6.0
 10     1       2     0.329682   5.0
 11     1       3     0.456953   8.0
 12     1       4     0.230314   1.0
 13     1       5     0.230314   2.0
 14     1       6     0.232325   4.0
 15     1       7     0.230314   3.0
 16     2       0     0.418334   7.0
 17     2       1     0.325578   6.0
 18     2       2     0.300596   5.0
 19     2       3     0.418334   8.0
 20     2       4     0.148714   1.0
 21     2       5     0.148714   2.0
 22     2       6     0.158609   4.0
 23     2       7     0.148714   3.0
 24     3       0     0.397146   7.0
 25     3       1     0.353656   6.0
 

In [649]:
Tagdistance

[    Qnno  Sentno  TagDistance  Rank
 0      0       0     0.378392   4.0
 1      0       1     0.349630   1.0
 2      0       2     0.416877   7.0
 3      0       3     0.378679   5.0
 4      0       4     0.371880   3.0
 5      0       5     0.391814   6.0
 6      0       6     0.350135   2.0
 7      1       0     0.397277   1.0
 8      1       1     0.450303   5.0
 9      1       2     0.482885   7.0
 10     1       3     0.448600   4.0
 11     1       4     0.458892   6.0
 12     1       5     0.438919   3.0
 13     1       6     0.412601   2.0
 14     2       0     0.206981   1.0
 15     2       1     0.305957   4.0
 16     2       2     0.360938   7.0
 17     2       3     0.292137   3.0
 18     2       4     0.343633   6.0
 19     2       5     0.333570   5.0
 20     2       6     0.242871   2.0
 21     3       0     0.397277   1.0
 22     3       1     0.450303   5.0
 23     3       2     0.482885   7.0
 24     3       3     0.448600   4.0
 25     3       4     0.458892   6.0
 

In [635]:
Tag_ans_sent[54]

[4, 4, 4, 4, 4]

In [412]:
len(Tagdistance)

121

In [776]:
tagdistance = [] 
tag_ans_sent = [] 

for i in range(0,1):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 
    

    for k in range(0,2):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])
        
        tag, tagans = tagdistancefun(processed_que_list,processed_sen_list)
        #w2vdistance.append(w2v)
        #w2v_ans_sent.append(w2vans) 
        tagdistance.append(tag)
        tag_ans_sent.append(tagans)
        

In [777]:
tagdistance

[    Qnno  Sentno  TagDistance  Rank
 0      0       0     0.378392   4.0
 1      0       1     0.349630   1.0
 2      0       2     0.416877   7.0
 3      0       3     0.378679   5.0
 4      0       4     0.371880   3.0
 5      0       5     0.391814   6.0
 6      0       6     0.350135   2.0
 7      1       0     0.397277   1.0
 8      1       1     0.450303   5.0
 9      1       2     0.482885   7.0
 10     1       3     0.448600   4.0
 11     1       4     0.458892   6.0
 12     1       5     0.438919   3.0
 13     1       6     0.412601   2.0
 14     2       0     0.206981   1.0
 15     2       1     0.305957   4.0
 16     2       2     0.360938   7.0
 17     2       3     0.292137   3.0
 18     2       4     0.343633   6.0
 19     2       5     0.333570   5.0
 20     2       6     0.242871   2.0
 21     3       0     0.397277   1.0
 22     3       1     0.450303   5.0
 23     3       2     0.482885   7.0
 24     3       3     0.448600   4.0
 25     3       4     0.458892   6.0
 

In [647]:
Tag_ans_sent[5]

[6, 6, 6, 6, 6]

In [648]:
tagdistance

[    Qnno  Sentno  TagDistance  Rank
 0      0       0     0.378392   4.0
 1      0       1     0.349630   1.0
 2      0       2     0.416877   7.0
 3      0       3     0.378679   5.0
 4      0       4     0.371880   3.0
 5      0       5     0.391814   6.0
 6      0       6     0.350135   2.0
 7      1       0     0.397277   1.0
 8      1       1     0.450303   5.0
 9      1       2     0.482885   7.0
 10     1       3     0.448600   4.0
 11     1       4     0.458892   6.0
 12     1       5     0.438919   3.0
 13     1       6     0.412601   2.0
 14     2       0     0.206981   1.0
 15     2       1     0.305957   4.0
 16     2       2     0.360938   7.0
 17     2       3     0.292137   3.0
 18     2       4     0.343633   6.0
 19     2       5     0.333570   5.0
 20     2       6     0.242871   2.0
 21     3       0     0.397277   1.0
 22     3       1     0.450303   5.0
 23     3       2     0.482885   7.0
 24     3       3     0.448600   4.0
 25     3       4     0.458892   6.0
 

##### Weighted distance #####

In [423]:
First_rank_list = [] 
Second_rank_list = []
Third_rank_list = [] 
Distance_list = [] 

for i in range(len(w2vdistance)):
    first_rank = [] 
    second_rank = [] 
    third_rank = [] 
    weighted_distance_list = [] 

    for a in range(len(weights)):

        weigDistance = pd.DataFrame((w2vdistance[i]['w2vDistance']*weights[a]) + (Tagdistance[i]['TagDistance']*(1-weights[a])),columns = ['WeigDistance']) 
        weigDistance['Qnno'] = w2vdistance[i]['Qnno'] 
        weigDistance['Sentno'] =  w2vdistance[i]['Sentno']
        weigDistance['WeiRank'] = weigDistance.groupby('Qnno')['WeigDistance'].rank(method = "first",ascending=True)
        first_rank.append(list(weigDistance[weigDistance['WeiRank']==1.0]['Sentno']))
        second_rank.append(list(weigDistance[weigDistance['WeiRank']==2.0]['Sentno']))
        third_rank.append(list(weigDistance[weigDistance['WeiRank']==3.0]['Sentno'])) 
        weighted_distance_list.append(weigDistance)


        
    First_rank_list.append(first_rank) 
    Second_rank_list.append(second_rank)
    Third_rank_list.append(third_rank)
    Distance_list.append(weighted_distance_list) 
    

In [455]:
First_rank_list[1]

[[1, 0, 8, 0, 7],
 [6, 0, 8, 0, 7],
 [2, 0, 8, 0, 7],
 [2, 9, 0, 0, 7],
 [2, 9, 0, 0, 7],
 [2, 9, 0, 0, 7],
 [2, 9, 0, 0, 7],
 [2, 9, 0, 0, 7],
 [2, 9, 0, 0, 7]]

1027

In [494]:
len(actualans_list)

121

##### Zero - One mapping ###### 

In [435]:
def toponemapping(list1,list2):
    ls = []
    for i in range(len(list1)):
        if (list1[i]==list2[i]): 
            ls.append(1) 
        else:
            ls.append(0)
            
    return ls

In [558]:
def fun(list1,list2,list3,list4):
    ls = []
    for i in range(len(list2)):
        if (list1[i]==list2[i]) or (list1[i]==list3[i]) or (list1[i]==list4[i]): 
            ls.append(1) 
        else:
            ls.append(0)
            
    return ls

In [438]:
toponemapping(actual_anssent[0],First_rank_list[0][0])

[1, 0, 0, 0, 0]

In [531]:
fun(First_rank_list[0][1],candans1_list[0],candans2_list[0]) 

[0, 0, 1, 0, 0]

In [559]:
weipredi_actual_mapping_list = []  
#toptworank_weig_list = []
weipredi_candidate_mapping_list = [] 

for p in range(len(weights)):

    first_rank_mapping = []
    #toptwo_rank_mapping = []
    topthree_rank_mapping = [] 
    for i in range(len(actualans_list)):
    #for i in range(0,75):

        first_rank_mapping.append(toponemapping(actual_anssent[i],First_rank_list[i][p]))
        #topthree = fun(First_rank_list[i][p],candans1_list[i],candans2_list[i])
        #toptwo_rank_mapping.append(toptwo)
        topthree_rank_mapping.append(fun(First_rank_list[i][p],actual_anssent[i],candans1_list[i],candans2_list[i])) 
        
        
    weipredi_actual_mapping_list.append(first_rank_mapping)
    #toptworank_weig_list.append(toptwo_rank_mapping)
    weipredi_candidate_mapping_list.append(topthree_rank_mapping)


   

In [540]:
print(candans1_list[75])

print(candans2_list[75])
print(First_rank_list[75])


[1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0]
[]
[[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1]]


In [553]:
len(weipredi_candidate_mapping_list[0])

121

In [484]:
topfirstrank1_weig_list[0]

[[1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0],
 [1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 0],
 [1, 0, 0, 1, 1],
 [1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 1],
 [1, 1, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 0],
 [1, 1, 1, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 0, 0],
 [0, 1, 0, 1, 1],
 [0, 0, 1, 1, 1],
 [0, 0, 1, 0, 1],
 [1, 1, 0, 1, 0],
 [0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 1],
 [0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 1, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0,

In [482]:
len(topfirstrank1_weig_list[0]) 

121

In [491]:
len(actual_anssent)

18896

In [514]:
First_rank_list

[[[5, 3, 3, 0, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3],
  [5, 3, 3, 3, 3]],
 [[1, 0, 8, 0, 7],
  [6, 0, 8, 0, 7],
  [2, 0, 8, 0, 7],
  [2, 9, 0, 0, 7],
  [2, 9, 0, 0, 7],
  [2, 9, 0, 0, 7],
  [2, 9, 0, 0, 7],
  [2, 9, 0, 0, 7],
  [2, 9, 0, 0, 7]],
 [[0, 0, 0, 3, 4],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5],
  [0, 0, 0, 3, 5]],
 [[2, 2, 2, 2, 2],
  [2, 0, 0, 2, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]],
 [[2, 2, 2, 2],
  [2, 2, 2, 2],
  [2, 2, 2, 1],
  [2, 0, 2, 1],
  [2, 0, 1, 1],
  [0, 0, 1, 1],
  [0, 0, 1, 1],
  [0, 0, 1, 1],
  [0, 0, 1, 1]],
 [[6, 6, 6, 6, 6],
  [6, 6, 6, 6, 6],
  [6, 6, 6, 6, 7],
  [6, 6, 6, 0, 7],
  [0, 6, 6, 0, 7],
  [0, 6, 1, 0, 7],
  [0, 6, 1, 0, 7],
  [0, 6, 11, 0, 7],
  [0, 6, 11, 0

In [510]:
candans1_list[0]

[0, 1, 2, 0, 5]

In [511]:
candans1_list[0]

[0, 1, 2, 0, 5]

In [544]:
def fun(list1,list2,list3):
    ls = []
    if (list3==[]):
        list3 = ['nan'] * len(list2)
    for i in range(len(list2)):
        if (list1[i]==list2[i]) or (list1[i]==list3[i]): 
            ls.append(1) 
        else:
            ls.append(0)
            
    return ls

In [555]:

weipredi_candidate_mapping_list[0]

[[0, 0, 1, 1, 0],
 [1, 1, 0, 1, 0],
 [0, 0, 1, 0, 1],
 [1, 1, 0, 1, 1],
 [0, 1, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 0, 1],
 [0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1],
 [0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0],
 [0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 0],
 [1, 1, 1, 1],
 [0, 0, 0, 1, 1],
 [0, 0, 1, 1, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 1, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0],
 [1, 0, 0, 1, 0],
 [1, 0, 1, 1, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 1, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 0, 0, 1,

In [560]:
top_firstrank_accuracy = [] 
candidate_accuracy = [] 
for a in range(len(topfirstrank1_weig_list)):
    topone_matching = sum([sum(l) for l in topfirstrank_weig_list[a]])/ sum([len(l) for l in actualans_list])
    candidate_matching = sum([sum(l) for l in weipredi_candidate_mapping_list[a]])/ sum([len(l) for l in actualans_list])
    top_firstrank_accuracy.append(topone_matching)
    candidate_accuracy.append(candidate_matching)


In [561]:
top_firstrank_accuracy

[0.33787731256085685,
 0.4079844206426485,
 0.46835443037974683,
 0.5199610516066212,
 0.5394352482960078,
 0.5569620253164557,
 0.5618305744888024,
 0.5705939629990263,
 0.5735150925024343]

In [562]:
candidate_accuracy

[0.6533592989289192,
 0.6893865628042843,
 0.7409931840311588,
 0.7740993184031159,
 0.7828627069133398,
 0.7887049659201558,
 0.7974683544303798,
 0.8033106134371957,
 0.801363193768257]

######################################################################################################################

In [312]:
paragraph_pointer = sqd_data.data[0]['paragraphs'] 
processed_sen_list, process_sen_token_list = preprocess_text(paragraph_pointer[0]['context'])
print(processed_sen_list)
processed_que_list = preprocess_question(paragraph_pointer[0]['qas'])
print()
print(processed_que_list)


['Architecturally, the school has a Catholic character.', "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.", 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".', 'Next to the Main Building is the Basilica of the Sacred Heart.', 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.', 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.', 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'What is in front of the Notre Dame Main Building?', 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'What is the Grotto at Notre Dame?', 'What sits on top of the Main Build

In [313]:
processed_sen_list

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [270]:
p = '-'.join(processed_sen_list)
p

'Architecturally, the school has a Catholic character.-Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary.-Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".-Next to the Main Building is the Basilica of the Sacred Heart.-Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.-It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.-At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [263]:
processed_sen_list[1][1]

't'

In [274]:
for i in (processed_sen_list):
    joinlist = '-'.join(processed_sen_list)
    for j in range(len(joinlist)):
        if (j == paragraph_pointer[0]['context'][188]) : 
            print(joinlist[j])
           
    
    

In [284]:
(paragraph_pointer[0]['context'])

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [285]:
paragraph_pointer[0]['context'][188] 

'a'

In [300]:
s = 'Appu is my friend. Ammu is good. all is well.'
len(s)      


45

In [308]:
s[30]

'd'

In [305]:
def fun(sent,index):
    count = 1
    for i in range(len(sent)):
        if(sent[i]=='.'):
            count +=1
        if(index==i):
            return count
    return -1
    

In [311]:
fun(s,32) 

3

In [216]:
sqd_data.data[0]['paragraphs']

[{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'qas': [{'answers': [{'answer_start': 515,
      'text': 'Saint Bernadette Soubirous'}],
    'id': '5733be284776f41900661182',
    'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
   {'answers': [{'answer_start': 188, 'text': 'a copper statue of 

In [773]:
Tagdistance = [] 
Tag_ans_sent = [] 

for i in range(0,1):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    for k in range(0,2):#no of paragraphs in a row of the df

        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])

        

        ls = [] 

        que_type_clean = qntypefun(processed_que_list)

        for m in range(len(que_type_clean)):

            map_que_ner = []
            map_que_ner.append(que_type_clean[m])
            print("Que_NER", " ", map_que_ner)
            for j in range(len(processed_sen_list)):   
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                



                map_ans_ner = []
                ans_ner = nlp(processed_sen_list[j])
                for ent in ans_ner.ents:
                    map_ans_ner.append('a_' + ent.label_)
                #print("Ans_NER", " ", map_ans_ner)
                unique_ner = ner_combination(map_que_ner, map_ans_ner)
                #print("Unique_NER", " ", unique_ner)
                #print()
                # distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
                d_1 = bag_of_words(map_que_ner, unique_ner)
                #print(d_1)

                d_2 = bag_of_words(map_ans_ner, unique_ner)
                #print(d_2)
                sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
                #print(sub_matrix_df)
                sub_matrix= sub_matrix_df.as_matrix()
                sub_matrix[np.isnan(sub_matrix)] = 0

                #print(sub_matrix)
                score = emd(d_1, d_2, sub_matrix)
                #print(processed_sen_list[j])
                #print('Score = ', score)
                #l1 = [m,j,score] 
                #ls.append(pd.DataFrame([m,j,score])
                #tagdis = pd.DataFrame(l1) 
                #tagdis['TagRank'] = tagdis.groupby('Qnno')['Tagdis'].rank(method = "first",ascending=True)
                #Tagdistance.append(ls)
                ls.append([m,j,score]) 
                
                
                
                df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
                df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

        Tagdistance.append(df)
        Tag_ans_sent.append(list(df[df['Rank']==1.0]['Sentno'])) 
    

Que_NER   ['whom']
Que_NER   ['what']
Que_NER   ['which']
Que_NER   ['what']
Que_NER   ['what']
Que_NER   ['when']
Que_NER   ['how']
Que_NER   ['what']
Que_NER   ['how']
Que_NER   ['what']


In [772]:
Tagdistance[0].head()

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.378392,4.0
1,0,1,0.349630,1.0
2,0,2,0.416877,7.0
3,0,3,0.378679,5.0
4,0,4,0.371880,3.0


In [ ]:
query = processed_que_list[0]
print('Query:')
print(query)
que_type  = question_type([query])
que_type_clean = question_type_extractor(que_type)
for i in range(len(processed_sen_list)):
    print()
    
    map_que_ner = []
    map_que_ner.append(que_type_clean[0])
    print("Que_NER", " ", map_que_ner)
    
    
    map_ans_ner = []
    ans_ner = nlp(processed_sen_list[i])
    for ent in ans_ner.ents:
        map_ans_ner.append('a_' + ent.label_)
    print("Ans_NER", " ", map_ans_ner)

    unique_ner = ner_combination(map_que_ner, map_ans_ner)
    print("Unique_NER", " ", unique_ner)
    print()
    distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
    
    d_1 = bag_of_words(map_que_ner, unique_ner)
#     print(d_1)
    d_2 = bag_of_words(map_ans_ner, unique_ner)
    
    sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
#     print(sub_matrix_df)
    sub_matrix= sub_matrix_df.as_matrix()
#     print(sub_matrix)
    
    score = emd(d_1, d_2, sub_matrix)
    print(processed_sen_list[i])
    print('Score = ', score)
    print()
    print("***************************************